In [2]:
import json
import datetime
from typing import Any, Callable, Set, Dict, List, Optional
from pydantic import BaseModel, Field

from azure.ai.projects.models import (
    ResponseFormatJsonSchema,
    ResponseFormatJsonSchemaType,
)


def fetch_weather(location: str) -> str:
    """
    Fetches the weather information for the specified location.

    :param location (str): The location to fetch weather for.
    :return: Weather information as a JSON string.
    :rtype: str
    """
    # In a real-world scenario, you'd integrate with a weather API.
    # Here, we'll mock the response.
    mock_weather_data = {"New York": "Sunny, 25°C", "London": "Cloudy, 18°C", "Tokyo": "Rainy, 22°C"}
    weather = mock_weather_data.get(location, "Weather data not available for this location.")
    weather_json = json.dumps({"weather": weather})
    return weather_json



def send_email(to: str, subject: str, body: str) -> str:
    """
    Sends an email with the specified subject and body to the given recipient.

    :param to (str): The recipient's email address.
    :param subject (str): The subject of the email.
    :param body (str): The body of the email.
    :return: Confirmation message.
    :rtype: s    # In a real-world scenario, you'd integrate with an email service.
    # Here, we'll mock the response.
    """
    return f"Email sent to {to} with subject '{subject}' and body '{body}'."


# Statically defined user functions for fast reference
user_functions: Set[Callable[..., Any]] = {
    fetch_weather,
    send_email,
}

In [43]:
import os
from azure.ai.projects import AIProjectClient
from azure.identity import DefaultAzureCredential
from azure.ai.projects.models import FunctionTool, ToolSet, FunctionDefinition
# from user_functions import user_functions # user functions which can be found in a user_functions.py file.

# Create an Azure AI Client from a connection string, copied from your Azure AI Foundry project.
# It should be in the format "<HostName>;<AzureSubscriptionId>;<ResourceGroup>;<HubName>"
# Customers need to login to Azure subscription via Azure CLI and set the environment variables

project_client = AIProjectClient.from_connection_string(
    credential=DefaultAzureCredential(),
    conn_str=os.environ["AZURE_AI_PROJECT_CONN_STR"],
)

# Initialize agent toolset with user functions
functions = FunctionTool(user_functions)
toolset = ToolSet()
toolset.add(functions)


class AgentResponseForWeather(BaseModel):
    location: str = Field(description="The location for which weather information was requested")
    weather_raw_json: str = Field(description="The raw JSON string containing weather data")
    weather_verbatim: str = Field(description="The human-readable weather information extracted from the JSON")

class EmailResponse(BaseModel):
    to: str = Field(default="(unknown)", description="The recipient's email address")
    subject: str = Field(description="The subject of the email")
    body: str = Field(description="The body of the email")
    status: str = Field(description="Status of the email sending operation")

class AgentResponse(BaseModel):
    weather: AgentResponseForWeather = Field(description="Weather information for the requested location")
    email: EmailResponse = Field(description="Email sending operation details")

agent = project_client.agents.create_agent(
    model="gpt-4o",
    name="my-weather-agent",
    instructions="""
    You are a weather bot. Use the provided functions to help answer questions.
    Ensure you have details you need from the user to execute the required task
    """,
    toolset=toolset,
    response_format=ResponseFormatJsonSchemaType(
        json_schema=ResponseFormatJsonSchema(
            name="weather_report",
            description="Weather report for a specific location",
            schema=AgentResponse.model_json_schema(),
        )
    )
)

print(f"Created agent, ID: {agent.id}")

Created agent, ID: asst_l422LnjorfowDPECCbhlV7E1


In [44]:
functions.definitions[0]

{'type': 'function', 'function': {'name': 'fetch_weather', 'description': 'Fetches the weather information for the specified location.', 'parameters': {'type': 'object', 'properties': {'location': {'type': 'string', 'description': 'The location to fetch weather for.'}}, 'required': ['location']}}}

In [45]:
# Create thread for communication
thread = project_client.agents.create_thread()
print(f"Created thread, ID: {thread.id}")

# Create message to thread
message = project_client.agents.create_message(
    thread_id=thread.id,
    role="user",
    content="Hello, send an email with the datetime and weather information in New York?",
)
print(f"Created message, ID: {message.id}")

Created thread, ID: thread_GF9x7YB7WFQNmTuuGUPjnPzM
Created message, ID: msg_aIzOY8tc7omDxkZpqByJoMI0


In [46]:
thread.as_dict()

{'id': 'thread_GF9x7YB7WFQNmTuuGUPjnPzM',
 'object': 'thread',
 'created_at': 1747267611,
 'metadata': {},
 'tool_resources': {}}

In [47]:
thread_messages = project_client.agents.list_messages(thread_id=thread.id, limit=10)

In [48]:
project_client.agents.get_message(
    thread_id=thread.id,
    message_id=message.id,)

{'id': 'msg_aIzOY8tc7omDxkZpqByJoMI0', 'object': 'thread.message', 'created_at': 1747267611, 'assistant_id': None, 'thread_id': 'thread_GF9x7YB7WFQNmTuuGUPjnPzM', 'run_id': None, 'role': 'user', 'content': [{'type': 'text', 'text': {'value': 'Hello, send an email with the datetime and weather information in New York?', 'annotations': []}}], 'attachments': [], 'metadata': {}}

In [49]:
# Create and process agent run in thread with tools
run = project_client.agents.create_and_process_run(thread_id=thread.id, agent_id=agent.id)
print(f"Run finished with status: {run.status}")

if run.status == "failed":
    print(f"Run failed: {run.last_error}")

# # Delete the agent when done
# project_client.agents.delete_agent(agent.id)
# print("Deleted agent")


# Fetch and log all messages
messages = project_client.agents.list_messages(thread_id=thread.id)
print(f"Messages: {messages}")

Run finished with status: completed
Messages: {'object': 'list', 'data': [{'id': 'msg_0A0SgDUAdn479Ix4bLazIsS3', 'object': 'thread.message', 'created_at': 1747267641, 'assistant_id': 'asst_l422LnjorfowDPECCbhlV7E1', 'thread_id': 'thread_GF9x7YB7WFQNmTuuGUPjnPzM', 'run_id': 'run_HtnMqejbePzjyKjYtfprysZV', 'role': 'assistant', 'content': [{'type': 'text', 'text': {'value': '{"email":{"to":"user@example.com","subject":"Weather Update: New York","body":"Hello,\\n\\nHere is the current date/time and weather information for New York.\\n\\nDate/Time: 2023-10-07T12:45:00Z\\nWeather: Sunny, 25°C\\n\\nRegards,\\nWeather Bot","status":"sent"},"weather":{"location":"New York","weather_raw_json":"{\\"weather\\": \\"Sunny, 25°C\\"}","weather_verbatim":"Sunny, 25°C"}}', 'annotations': []}}], 'attachments': [], 'metadata': {}}, {'id': 'msg_aIzOY8tc7omDxkZpqByJoMI0', 'object': 'thread.message', 'created_at': 1747267611, 'assistant_id': None, 'thread_id': 'thread_GF9x7YB7WFQNmTuuGUPjnPzM', 'run_id': Non

In [50]:
for d in messages.data:
    # print(d)
    print(d.role)
    print(d.content[0].text)

MessageRole.AGENT
{'value': '{"email":{"to":"user@example.com","subject":"Weather Update: New York","body":"Hello,\\n\\nHere is the current date/time and weather information for New York.\\n\\nDate/Time: 2023-10-07T12:45:00Z\\nWeather: Sunny, 25°C\\n\\nRegards,\\nWeather Bot","status":"sent"},"weather":{"location":"New York","weather_raw_json":"{\\"weather\\": \\"Sunny, 25°C\\"}","weather_verbatim":"Sunny, 25°C"}}', 'annotations': []}
MessageRole.USER
{'value': 'Hello, send an email with the datetime and weather information in New York?', 'annotations': []}


In [51]:
last_message = messages.get_last_message_by_role("assistant")

In [52]:
last_message

{'id': 'msg_0A0SgDUAdn479Ix4bLazIsS3', 'object': 'thread.message', 'created_at': 1747267641, 'assistant_id': 'asst_l422LnjorfowDPECCbhlV7E1', 'thread_id': 'thread_GF9x7YB7WFQNmTuuGUPjnPzM', 'run_id': 'run_HtnMqejbePzjyKjYtfprysZV', 'role': 'assistant', 'content': [{'type': 'text', 'text': {'value': '{"email":{"to":"user@example.com","subject":"Weather Update: New York","body":"Hello,\\n\\nHere is the current date/time and weather information for New York.\\n\\nDate/Time: 2023-10-07T12:45:00Z\\nWeather: Sunny, 25°C\\n\\nRegards,\\nWeather Bot","status":"sent"},"weather":{"location":"New York","weather_raw_json":"{\\"weather\\": \\"Sunny, 25°C\\"}","weather_verbatim":"Sunny, 25°C"}}', 'annotations': []}}], 'attachments': [], 'metadata': {}}

In [53]:
type(last_message.content)
type(last_message.content[0])

azure.ai.projects.models._models.MessageTextContent

In [54]:
msg_text_content = last_message.content[0]

In [55]:
msg_text_content.text['value']

'{"email":{"to":"user@example.com","subject":"Weather Update: New York","body":"Hello,\\n\\nHere is the current date/time and weather information for New York.\\n\\nDate/Time: 2023-10-07T12:45:00Z\\nWeather: Sunny, 25°C\\n\\nRegards,\\nWeather Bot","status":"sent"},"weather":{"location":"New York","weather_raw_json":"{\\"weather\\": \\"Sunny, 25°C\\"}","weather_verbatim":"Sunny, 25°C"}}'

In [56]:
wreport = AgentResponse.model_validate_json(msg_text_content.text['value'])

In [57]:
import json

print(json.dumps(wreport.model_dump(), indent=2))

{
  "weather": {
    "location": "New York",
    "weather_raw_json": "{\"weather\": \"Sunny, 25\u00b0C\"}",
    "weather_verbatim": "Sunny, 25\u00b0C"
  },
  "email": {
    "to": "user@example.com",
    "subject": "Weather Update: New York",
    "body": "Hello,\n\nHere is the current date/time and weather information for New York.\n\nDate/Time: 2023-10-07T12:45:00Z\nWeather: Sunny, 25\u00b0C\n\nRegards,\nWeather Bot",
    "status": "sent"
  }
}


In [58]:
message = project_client.agents.create_message(
    thread_id=thread.id,
    role="user",
    content="send the message to name@email.com",
)

In [59]:
run = project_client.agents.create_and_process_run(thread_id=thread.id, agent_id=agent.id)
print(f"Run finished with status: {run.status}")

if run.status == "failed":
    print(f"Run failed: {run.last_error}")

messages = project_client.agents.list_messages(thread_id=thread.id)
print(f"Messages: {messages}")

Run finished with status: completed
Messages: {'object': 'list', 'data': [{'id': 'msg_ImdIMoXuRKa0lrlwpxXjxCbR', 'object': 'thread.message', 'created_at': 1747267733, 'assistant_id': 'asst_l422LnjorfowDPECCbhlV7E1', 'thread_id': 'thread_GF9x7YB7WFQNmTuuGUPjnPzM', 'run_id': 'run_72cfCN6et4HcsankFGn9vJP3', 'role': 'assistant', 'content': [{'type': 'text', 'text': {'value': '{"email":{"to":"name@email.com","subject":"Weather Update: New York","body":"Hello,\\n\\nHere is the current date/time and weather information for New York.\\n\\nDate/Time: 2023-10-07T12:45:00Z\\nWeather: Sunny, 25°C\\n\\nRegards,\\nWeather Bot","status":"sent"},"weather":{"location":"New York","weather_raw_json":"{\\"weather\\": \\"Sunny, 25°C\\"}","weather_verbatim":"Sunny, 25°C"}}', 'annotations': []}}], 'attachments': [], 'metadata': {}}, {'id': 'msg_cDPc3etmmn7oajmL48ekTMea', 'object': 'thread.message', 'created_at': 1747267726, 'assistant_id': None, 'thread_id': 'thread_GF9x7YB7WFQNmTuuGUPjnPzM', 'run_id': None,

In [60]:
for d in messages.data:
    # print(d)
    print(d.role)
    print(d.content[0].text)

MessageRole.AGENT
{'value': '{"email":{"to":"name@email.com","subject":"Weather Update: New York","body":"Hello,\\n\\nHere is the current date/time and weather information for New York.\\n\\nDate/Time: 2023-10-07T12:45:00Z\\nWeather: Sunny, 25°C\\n\\nRegards,\\nWeather Bot","status":"sent"},"weather":{"location":"New York","weather_raw_json":"{\\"weather\\": \\"Sunny, 25°C\\"}","weather_verbatim":"Sunny, 25°C"}}', 'annotations': []}
MessageRole.USER
{'value': 'send the message to name@email.com', 'annotations': []}
MessageRole.AGENT
{'value': '{"email":{"to":"user@example.com","subject":"Weather Update: New York","body":"Hello,\\n\\nHere is the current date/time and weather information for New York.\\n\\nDate/Time: 2023-10-07T12:45:00Z\\nWeather: Sunny, 25°C\\n\\nRegards,\\nWeather Bot","status":"sent"},"weather":{"location":"New York","weather_raw_json":"{\\"weather\\": \\"Sunny, 25°C\\"}","weather_verbatim":"Sunny, 25°C"}}', 'annotations': []}
MessageRole.USER
{'value': 'Hello, send 